# Step 3: Train and evaluate the Calorimetric method

In [2]:
distances = [15, 35]
doubleplanes = [8, 12, 30]
energies = [200, 600, 1000]
erels = [100, 500]
neutrons = [1, 2, 3, 4, 5, 6]
physicss = ["bert", "bic", "inclxx"]
scenarios = ["air", "vacuum"]

## a) Train calorimetric method for all settings

In [1]:
import os
import wurlitzer
import joblib
import ROOT

Welcome to JupyROOT 6.16/00


In [3]:
ROOT.ROOT.EnableThreadSafety()
ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")
ROOT.FairLogger.GetLogger().SetLogScreenLevel("WARNING")
ROOT.gROOT.SetBatch(True)

In [4]:
def calibr(distance, doubleplane, energy, erel, nmax, physics, scenario):
    filepattern = "output/%s-%s/%dm_%ddp_%dAMeV_%dkeV_%dn.%s.root"
    outfile = filepattern % (physics, scenario, distance, doubleplane, energy, erel, nmax, "ncut")
    
    if os.path.exists(outfile):
        os.remove(outfile)
    
    # Write all output to a log file
    logfile = outfile.replace(".root", ".log")
    with open(logfile, "w") as log, wurlitzer.pipes(stdout=log, stderr=wurlitzer.STDOUT):
        cal = ROOT.Neuland.Neutron2DCalibr(nmax)
        for neutron in range(1, nmax + 1):
            digifile = filepattern % (physics, scenario, distance, doubleplane, energy, erel, neutron, "digi")
            cal.AddClusterFile(digifile)

        # Starting parameters (double val, double step, double lower, double upper)
        vslope = ROOT.std.vector('double')()
        vslope += [0.04, 0.001, 0.001, 10]
        vdistance = ROOT.std.vector('double')()
        vdistance += [energy / 20., energy / 100., energy / 100., energy]
        vdist_off = ROOT.std.vector('double')()
        vdist_off += [3, 0.5, 3, 6]

        # Create Cuts
        cal.Optimize(vslope, vdistance, vdist_off);

        # Write the output files (root, dat, pdf)
        cal.WriteParameterFile(outfile)
        o = ROOT.std.ofstream(outfile.replace(".root", ".dat"))
        cal.Print(o)
        cal.Draw(outfile.replace(".root", ".pdf"))
        del cal

In [5]:
joblib.Parallel(n_jobs=-1, backend="multiprocessing", verbose=1)(
    joblib.delayed(calibr)(
        distance=distance,
        doubleplane=doubleplane,
        energy=energy,
        erel=erel,
        nmax=neutron,
        physics=physics,
        scenario=scenario,
    )
    for distance in distances
    for energy in energies
    for doubleplane in doubleplanes
    for neutron in neutrons
    for erel in erels
    for physics in physicss
    for scenario in scenarios
);

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:   51.4s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1138 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 1296 out of 1296 | elapsed:  7.6min finished


''

## b) Apply cuts to a setting and evaluate performance

In [6]:
import numpy as np
import pandas as pd

For each event, apply the cuts and list the predicted multiplicity along with the number of incoming neutrons, the number of primary points, etc.

In [7]:
def neuland_calorimetric_data(distance, doubleplane, energy, erel, nmax, physics, scenario):
    filepattern = "output/%s-%s/%dm_%ddp_%dAMeV_%dkeV_%dn.%s.root"
    ncutfile = filepattern % (physics, scenario, distance, doubleplane, energy, erel, nmax, "ncut")

    file = ROOT.TFile.Open(ncutfile)
    para = file.Get("R3BNeulandNeutron2DPar")

    data = []
    num_ex = 0
    for neutron in range(1, nmax + 1):
        digifile = filepattern % (physics, scenario, distance, doubleplane, energy, erel, neutron, "digi")
        tfile = ROOT.TFile.Open(digifile)
        ttree = tfile.Get("evt")
        for event in ttree:
            # nPN: Number of incoming primary neutrons
            nIn = neutron
            # nPP: Number of primary neutrons with an energy deposition in NeuLAND
            nPP = event.NeulandPrimaryPoints.GetEntries()
            # nPH: Number of hits that correspond to a energy deposition of a primary neutron
            nPH = event.NeulandPrimaryHits.GetEntries()
            # nHits: Number of hits
            num_hits = event.NeulandHits.GetEntries()
            # nClus: Number of clusters
            num_clusters = event.NeulandClusters.GetEntries()
            # Edep: Total deposited (detected) energy
            try:
                edep = round(sum([hit.GetE() for hit in event.NeulandHits]))
            except:
                num_ex += 1
                edep = 0

            # Use calibrated calorimetric method (R3BNeulandNeutron2DPar) from the parameter file
            # to predict the multiplicity
            mult = para.GetNeutronMultiplicity(edep, num_clusters)

            data.append([nIn, nPP, nPH, num_hits, num_clusters, edep, mult])
    datapd = pd.DataFrame(data)
    datapd.columns = ["nIn", "nPP", "nPH", "num_hits", "num_cluster", "edep", "mult"]
    return datapd

In [8]:
distance = 15
doubleplane = 30
erel = 500
nmax = 5
physics = "inclxx"
scenario = "air"

data = {
    energy: neuland_calorimetric_data(
        distance, doubleplane, energy, erel, nmax, physics, scenario
    )
    for energy in [200, 600, 1000]
}

pd.options.display.max_rows = 20
display(data[600])

,nIn,nPP,nPH,num_hits,num_cluster,edep,mult
0,1,1,1,11,8,198,1
1,1,1,1,13,3,270,1
2,1,1,1,14,7,339,1
3,1,1,1,19,2,355,1
4,1,1,1,20,4,316,1
...,...,...,...,...,...,...,...
49995,5,4,4,71,19,1258,4
49996,5,5,5,53,26,1050,4
49997,5,5,5,75,20,1487,5
49998,5,5,5,84,28,1507,5


Annotate the image created by the calibration. Ready for use in the paper.

In [9]:
import shutil
import subprocess

energy = 600
ncutpdf = "%dm_%ddp_%dAMeV_%dkeV_%dn.ncut.pdf" % (
    distance,
    doubleplane,
    energy,
    erel,
    nmax,
)
shutil.copyfile("output/%s-%s/%s" % (physics, scenario, ncutpdf), "paper/" + ncutpdf)
subprocess.call(["lualatex", "calibr.tex"], cwd="paper", stdout=subprocess.DEVNULL)

0

Example neutron separation matrices and balanced accuracy score

In [10]:
from sklearn.metrics import (
    balanced_accuracy_score,
    confusion_matrix,
    plot_confusion_matrix,
)

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

out = []
for energy, df in data.items():
    print(energy)
    y_true = df[["nIn"]].values.ravel()
    y_pred = df[["mult"]].values.ravel()

    bac = balanced_accuracy_score(y_true, y_pred)
    print(bac)
    # cm = confusion_matrix(y_true, y_pred, labels=range(0, 6))
    # print(np.swapaxes(cm, 0, 1))
    cmrel = confusion_matrix(y_true, y_pred, labels=range(0, 6), normalize="true")
    cmrel = np.swapaxes(cmrel, 0, 1)
    print(cmrel)
    out.append([energy, bac, cmrel])

200
0.63498
[[0.    0.115 0.009 0.001 0.    0.   ]
 [0.    0.842 0.213 0.03  0.002 0.   ]
 [0.    0.043 0.682 0.267 0.057 0.007]
 [0.    0.    0.097 0.577 0.308 0.079]
 [0.    0.    0.    0.123 0.492 0.332]
 [0.    0.    0.    0.001 0.141 0.583]]
600
0.64896
[[0.    0.056 0.003 0.    0.    0.   ]
 [0.    0.813 0.137 0.014 0.001 0.   ]
 [0.    0.131 0.734 0.236 0.042 0.006]
 [0.    0.    0.127 0.65  0.334 0.084]
 [0.    0.    0.    0.099 0.541 0.403]
 [0.    0.    0.    0.    0.081 0.507]]
1000
0.6693200000000001
[[0.    0.04  0.001 0.    0.    0.   ]
 [0.    0.811 0.109 0.011 0.001 0.   ]
 [0.    0.149 0.769 0.211 0.035 0.005]
 [0.    0.    0.121 0.697 0.332 0.082]
 [0.    0.    0.    0.082 0.577 0.421]
 [0.    0.    0.    0.    0.054 0.492]]


/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


Create a Latex file with the multiplicity matrices for use in the paper

In [11]:
def to_tabular(energy, bac, cm):
    text = """\\resizebox{0.32\\textwidth}{!}{
\\begin{tabular}{cc|S[table-format=2]S[table-format=2]S[table-format=2]S[table-format=2]S[table-format=2]}
\\toprule
\\multicolumn{2}{c|}{""" + str(energy) + """} & \\multicolumn{5}{c}{generated} \\\\
\\multicolumn{2}{c|}{""" + str("MeV") + """} & 1 & 2 & 3 & 4 & 5 \\\\
\\midrule
\\multirow{6}{*}{\\rotatebox[origin=c]{90}{detected}}
"""
    for i, a in enumerate(cm):
        text += " & " +  str(i)
        # text += " & ".join([ for x in a[1:]])
        for j, x in enumerate(a[1:]):
            if j == i - 1: 
                text += " & \\textbf{" + str(int(round(x*100))) + "}"
            else:
                text += " & " + str(int(round(x*100)))
        text += " \\\\\n"
    text += """\\bottomrule
\\end{tabular}}"""
    return text

In [12]:
def to_full_latex(data):
    text = """\\documentclass{{scrartcl}}
\\usepackage{booktabs}
\\usepackage{siunitx}
\\usepackage{multirow}
\\usepackage{graphicx}
\\begin{document}
\\begin{table}
"""
    text += f"""\\captionabove[Neutron separation matrices for multiplicities of 1 to {nmax} neutrons]{{
Neutron separation matrices for multiplicities of 1 to {nmax} neutrons.
Columns display the neutron multiplicity simulated, rows the neutron multiplicity derived from the calorimetric neutron tracking algorithm.
Values are given in percent.
Neutrons were simulated with 200 (left), 600 (center) and 1000 MeV (right matrix).
NeuLAND with {doubleplane} doubleplanes was located at a distance of {distance} m to the target.
Neutrons were generated with a relative energy of {erel} keV with respect to a medium heavy projectile fragment.
The distance between target and NeuLAND was filled with {scenario}.
Simulated with Geant4 using the \\texttt{{QGSP\\_{physics.upper()}\\_HP}} physics list.
\\label{{tab:neutron-separation-matrices}}}}
"""
    for o in out:
        text += to_tabular(*o) + "\n"
    text += """\\end{table}
\\end{document}
"""
    return text

In [13]:
latex = to_full_latex(out)
print(latex)
with open("paper/mult.tex", "w") as f:
    f.write(latex)
subprocess.call(["lualatex", "mult.tex"], cwd="paper", stdout=subprocess.DEVNULL)

\documentclass{{scrartcl}}
\usepackage{booktabs}
\usepackage{siunitx}
\usepackage{multirow}
\usepackage{graphicx}
\begin{document}
\begin{table}
\captionabove[Neutron separation matrices for multiplicities of 1 to 5 neutrons]{
Neutron separation matrices for multiplicities of 1 to 5 neutrons.
Columns display the neutron multiplicity simulated, rows the neutron multiplicity derived from the calorimetric neutron tracking algorithm.
Values are given in percent.
Neutrons were simulated with 200 (left), 600 (center) and 1000 MeV (right matrix).
NeuLAND with 30 doubleplanes was located at a distance of 15 m to the target.
Neutrons were generated with a relative energy of 500 keV with respect to a medium heavy projectile fragment.
The distance between target and NeuLAND was filled with air.
Simulated with Geant4 using the \texttt{QGSP\_INCLXX\_HP} physics list.
\label{tab:neutron-separation-matrices}}
\resizebox{0.32\textwidth}{!}{
\begin{tabular}{cc|S[table-format=2]S[table-format=2]S[table-

0